# Dataset de linhas de ônibus

In [ ]:
# Importa dependências
import time
from datetime import datetime, timedelta
from io import StringIO
import pandas as pd
import os
from dotenv import load_dotenv
# Importa o módulo api_olhovivo e autentica no Olho Vivo API
import sys
sys.path.insert(0, '../modules')
import api_olhovivo

def autenticar_api():
    load_dotenv()  # Carrega as variáveis de ambiente do arquivo .env
    # Acesse o token de API do arquivo .env
    api_token = os.getenv("API_TOKEN")
    api = api_olhovivo.OlhoVivoAPI(api_token)
    return api

def run_posicao_veiculos_at_interval(interval, duration, max_retries=6, max_failures=3, backoff_factor=2):
    start_time = datetime.now()
    end_time = start_time
    df = pd.DataFrame(columns=['hr', 'c', 'cl', 'sl', 'lt0', 'lt1', 'qv', 'p', 'a', 'ta', 'py', 'px'])
    api = autenticar_api()  # Autenticar no início
    consecutive_failures = 0  # Contador de falhas consecutivas

    while (end_time - start_time).seconds < duration * 60:
        result = api.posicao_veiculos()
        if result is None:
            consecutive_failures += 1
            if consecutive_failures >= max_failures:  # Tentar novamente após 3 falhas
                num_retries = 0
                while result is None and num_retries < max_retries:
                    wait_time = backoff_factor ** num_retries  # Cálculo de backoff exponencial
                    time.sleep(wait_time)
                    api = autenticar_api()
                    result = api.posicao_veiculos()
                    num_retries += 1
                consecutive_failures = 0  # Reiniciar contador se a nova tentativa for bem-sucedida
        else:
            consecutive_failures = 0  # Reiniciar contador se a requisição for bem-sucedida

        if result is not None:
            num_retries = 0
            
            # Extrair hora da consulta
            hora_consulta = result['hr']

            # Lista para armazenar informações dos veículos
            lista_veiculos = []

            # Iterar sobre as linhas e veículos
            for linha in result['l']:
                codigo_linha = linha['c']
                codigo_linha_circular = linha['cl']
                sentido_linha = linha['sl']
                letreiro_0 = linha['lt0']
                letreiro_1 = linha['lt1']
                quantidade_veiculos = linha['qv']

                for veiculo in linha['vs']:
                    prefixo = veiculo['p']
                    acessivel = veiculo['a']
                    data_hora = veiculo['ta']
                    latitude = veiculo['py']
                    longitude = veiculo['px']

                    # Adicionar informações à lista
                    lista_veiculos.append({
                        'hr': hora_consulta, 
                        'c': codigo_linha, 
                        'cl': codigo_linha_circular,
                        'sl': sentido_linha,
                        'lt0': letreiro_0,
                        'lt1': letreiro_1,
                        'qv': quantidade_veiculos,
                        'p': prefixo,
                        'a': acessivel,
                        'ta': data_hora,
                        'py': latitude,
                        'px': longitude
                    })

            # Converter lista em DataFrame
            df = pd.concat([df, pd.DataFrame(lista_veiculos)])
        time.sleep(interval * 60)
        end_time = datetime.now()
    return df

def formatar_datahora(datahora_obj):
    """ Formata um objeto Timestamp do Pandas para o formato AAAA-MM-DD hh:mm:ss subtraindo 3 horas. """
    return (datahora_obj - pd.Timedelta(hours=3)).strftime("%Y-%m-%d %H:%M:%S")

def arredondar_meio(numero):
    """ Arredonda um número para duas casas decimais usando arredondamento usual. """
    return round(numero, 4)

import pandas as pd


def dataframe_to_textfile(dataframe, filename_base, columns):
    """
    Salva colunas específicas de um DataFrame em dois arquivos de texto: um com separador de espaço e outro com formatação específica.

    Argumentos:
        dataframe: O DataFrame do Pandas que contém os dados.
        filename_base: O nome base para os arquivos de saída (sem sufixos).
        columns: Uma lista de nomes de colunas a serem salvas nos arquivos.

    Levanta:
        ValueError: Se alguma das colunas especificadas estiver faltando no DataFrame.
    """

    # Verificação de erro para colunas ausentes
    if not all(col in dataframe.columns for col in columns):
        ausentes = [col for col in columns if col not in dataframe.columns]
        raise ValueError(f"Colunas ausentes: {', '.join(ausentes)}")

    # Etapas de processamento de dados
    dataframe['ta'] = pd.to_datetime(dataframe['ta'])
    dataframe.sort_values(by='ta', inplace=True)
    dataframe = dataframe.drop_duplicates(subset=['ta', 'p'], keep='last')

    # Certifique-se de que "c" está incluído na lista de colunas
    if "c" not in columns:
        columns.append("c")

    # Selecionar e formatar colunas (se necessário)
    ordem_desejada = ["ta", "px", "py", "p", "c", "lt0"]
    df_subset = dataframe[ordem_desejada].copy()

    for col in ordem_desejada:
        if col == "ta":
            df_subset[col] = df_subset[col].apply(formatar_datahora)
        elif col == "c":
            df_subset[col] = df_subset[col].astype(str)
        elif col in ("py", "px"):
            df_subset[col] = df_subset[col].apply(arredondar_meio)

    # Função para adicionar ponto e vírgula no final de cada linha
    def save_with_semicolon(filename, sep):
        with open(filename, 'w') as file:
            for line in df_subset.to_csv(index=False, sep=sep).split('\n'):
                if line.strip():
                    file.write(line + ';\n')

    # Função para numerar linhas e formatar colunas com espaço como separador
    def save_with_custom_format(filename):
        with open(filename, 'w') as file:
            for idx, row in enumerate(df_subset.itertuples(index=False), 1):
                formatted_row = ' '.join(map(str, row))
                file.write(f"{idx}, {formatted_row};\n")

    # Gerar o primeiro arquivo (separador de espaço)
    filename_pd = filename_base + "_pd.txt"
    save_with_semicolon(filename_pd, sep="\t")

    # Gerar o segundo arquivo (numerado, colunas separadas por espaço, linhas terminadas com ponto e vírgula)
    filename_max = filename_base + "_max.txt"
    save_with_custom_format(filename_max)


def schedule_function(function, run_date, run_time):
  """
  Agenda a execução de uma função em um horário e data específicos e retorna o valor retornado pela função.

  Args:
      function: A função a ser executada.
      run_date: A data para executar a função, no formato "DD/MM/YYYY".
      run_time: O horário para executar a função, no formato "HH:MM".

  Returns:
      O valor retornado pela função executada.
  """

  while True:
    now = datetime.now()
    current_date = now.strftime("%d/%m/%Y")
    current_time = now.strftime("%H:%M")
    
    if current_date == run_date and current_time == run_time:
      return function() # Retorna o valor retornado pela função
    
    time.sleep(1) # Aguarda 1 segundo antes de verificar novamente

In [ ]:
df = run_posicao_veiculos_at_interval(0.15, 0.25)
dataframe_to_textfile(df, "dados_rush", ["ta", "py", "px", "p", "lt0"])
df

In [ ]:
df1 = schedule_function(lambda: run_posicao_veiculos_at_interval(0.2, 120), "11/05/2024", "02:40")
df1.to_json('rush_da_madrudaga_sabado.json', orient='records', indent=4)
dataframe_to_textfile(df1, "rush_da_madrugada_sabado_2024-05-11.txt", ["ta", "py", "px", "p", "lt0"])

In [ ]:
df3 = pd.read_json('rush_da_madrudaga_sabado.json')
dataframe_to_textfile(df3, "rush_da_madrugada_sabado_2024-05-11.txt", ["ta", "py", "px", "p", "lt0"])
df3

# Dataset de Velocidades Médias

In [ ]:
import re
from zipfile import ZipFile
from bs4 import BeautifulSoup
import html

def extract_kml_from_kmz(kmz_pathway):
    """
    Extrai o arquivo KML de um arquivo KMZ.

    Args:
        kmz_pathway: O caminho para o arquivo KMZ.

    Returns:
        O conteúdo do arquivo KML como uma string.
    """
    try:
        with ZipFile(kmz_pathway, 'r') as kmz:
            # List all files in the KMZ archive
            unzip_file_list = kmz.namelist()
            # Get the KML file
            kml_file_name = 'TBC.kml' if 'TBC.kml' in unzip_file_list else unzip_file_list[0]
            # Open and read the KML file
            with kmz.open(kml_file_name, 'r') as kml_file:
                kml_content = kml_file.read().decode('utf-8')
            
            return kml_content
    except Exception as e:
        raise e

def parse_description(description):
    """
    Analisa a string de descrição em um dicionário estruturado, substituindo valores vazios, "0" e "--" por "-1".
    Garante que todas as chaves estejam presentes no dicionário final.

    Argumentos:
        description: A string de descrição de um Placemark.

    Retorna:
        Um dicionário com a descrição estruturada.
    """
    description_dict = {}
    parts = description.split(';')

    # Função para tratar valores especiais
    def tratar_valor(value):
        if value in {"", "0", "--"}:
            return -1
        return value

    # Map description parts to dictionary keys
    for i in range(0, len(parts) - 1, 2):
        key = parts[i].strip().rstrip(':')
        value = parts[i + 1].strip()
        value = tratar_valor(value)

        if key == 'Trecho':
            description_dict['trecho'] = value
        elif key == 'Referência':
            description_dict['referencia'] = value
        elif key == 'Extensão (metros)':
            try:
                description_dict['extensao_metros'] = int(value.replace('.', '')) if isinstance(value, str) else value
            except ValueError:
                description_dict['extensao_metros'] = tratar_valor(value)  # Preserve original value if not a number
        elif key == 'Velocidade média do trecho':
            try:
                description_dict['velocidade_media_trecho_kmh'] = int(value.split()[0]) if isinstance(value, str) else value
            except ValueError:
                description_dict['velocidade_media_trecho_kmh'] = tratar_valor(value)  # Preserve original value if not a number
        elif key == 'Tempo médio do percurso':
            try:
                if isinstance(value, str):
                    hours, minutes = map(int, value.split('h'))
                    total_minutes = hours * 60 + minutes
                    description_dict['tempo_medio_percurso_minutos'] = total_minutes
                else:
                    description_dict['tempo_medio_percurso_minutos'] = value
            except ValueError:
                description_dict['tempo_medio_percurso_minutos'] = tratar_valor(value)  # Preserve original value if not a valid time format
        elif key == 'Velocidade média do corredor':
            try:
                description_dict['velocidade_media_corredor_kmh'] = int(value.split()[0]) if isinstance(value, str) else value
            except ValueError:
                description_dict['velocidade_media_corredor_kmh'] = tratar_valor(value)  # Preserve original value if not a number

    # Garantir que todas as chaves estejam presentes
    required_keys = ['trecho', 'referencia', 'extensao_metros', 'velocidade_media_trecho_kmh', 'tempo_medio_percurso_minutos', 'velocidade_media_corredor_kmh']
    for key in required_keys:
        if key not in description_dict:
            description_dict[key] = -1

    return description_dict


def read_kml_content(kml_content):
    """
    Extrai dados de texto das descrições e coordenadas de todos os Placemarks de um arquivo KML, preservando o texto e removendo tags e entidades HTML.
    Usa o ponto e vírgula como separador entre diferentes descrições.

    Args:
        kml_content: O conteúdo do arquivo KML como uma string.

    Returns:
        Uma lista de dicionários com descrições e coordenadas para cada Placemark.
    """
    placemarks = []
    for placemark in kml_content.split('<Placemark'):
        if not placemark.strip():
            continue
        
        placemark_data = {}
        
        # Extract description
        description_start = placemark.find('<description>')
        description_end = placemark.find('</description>')
        if description_start != -1 and description_end != -1:
            description = placemark[description_start + len('<description>'): description_end]
            description = html.unescape(description)
            soup = BeautifulSoup(description, 'html.parser')
            text_only_description = soup.get_text(separator=';', strip=True)
            structured_description = parse_description(text_only_description)
            placemark_data.update(structured_description)  # Merge description keys into placemark_data

        # Extract coordinates
        coordinates_start = placemark.find('<coordinates>')
        coordinates_end = placemark.find('</coordinates>')
        if coordinates_start != -1 and coordinates_end != -1:
            coordinates = placemark[coordinates_start + len('<coordinates>'): coordinates_end]
            coordinates = coordinates.strip().split()
            coordinates = [coord.split(',')[:2] for coord in coordinates]  # Discarding the third value
            placemark_data['coordenadas'] = coordinates
        else:
            placemark_data['coordenadas'] = -1

        placemarks.append(placemark_data)
    
    return placemarks

def exportar_velocidades_medias_Pd(data, filename):
    with open(filename, 'w') as file:
        # Write the header
        headers = ['trecho', 'referencia', 'extensao_metros', 'velocidade_media_trecho_kmh', 'tempo_medio_percurso_minutos', 'velocidade_media_corredor_kmh', 'coordenadas']
        file.write(' '.join(headers) + '\n')
        
        for placemark in data:
            trecho = f"\"{placemark['trecho']}\"" if placemark['trecho'] != -1 else "-1"
            referencia = f"\"{placemark['referencia']}\"" if placemark['referencia'] != -1 else "-1"
            extensao = placemark['extensao_metros']
            velocidade_trecho = placemark['velocidade_media_trecho_kmh']
            tempo_percurso = placemark['tempo_medio_percurso_minutos']
            velocidade_corredor = placemark['velocidade_media_corredor_kmh']

            # Agrupar coordenadas em pares
            if placemark['coordenadas'] != -1:
                coordenadas_list = placemark['coordenadas']
                coordenadas_pares = [f"{coord[0]} {coord[1]}" for coord in coordenadas_list]
                coordenadas = ','.join(coordenadas_pares)
            else:
                coordenadas = "-1"

            file.write(f"{trecho} {referencia} {extensao} {velocidade_trecho} {tempo_percurso} {velocidade_corredor} {coordenadas}\n")

def exportar_velocidades_medias_Max(data, filename):
    with open(filename, 'w') as file:
        for idx, placemark in enumerate(data, start=1):
            trecho = f"\"{placemark['trecho']}\"" if placemark['trecho'] != -1 else "-1"
            referencia = f"\"{placemark['referencia']}\"" if placemark['referencia'] != -1 else "-1"
            extensao = placemark['extensao_metros']
            velocidade_trecho = placemark['velocidade_media_trecho_kmh']
            tempo_percurso = placemark['tempo_medio_percurso_minutos']
            velocidade_corredor = placemark['velocidade_media_corredor_kmh']

            # Agrupar coordenadas em pares
            if placemark['coordenadas'] != -1:
                coordenadas_list = placemark['coordenadas']
                coordenadas_pares = [f"{coord[0]} {coord[1]}" for coord in coordenadas_list]
                coordenadas = ','.join(coordenadas_pares)
                coordenadas = f"\"{coordenadas}\""
            else:
                coordenadas = "\"-1\""

            file.write(f"{idx}, {trecho} {referencia} {extensao} {velocidade_trecho} {tempo_percurso} {velocidade_corredor} {coordenadas};\n")

def exportar_datasets(data, filename_base):
    exportar_velocidades_medias_Pd(data, f"{filename_base}_Pd.txt")
    exportar_velocidades_medias_Max(data, f"{filename_base}_Max.txt")





In [ ]:
api = autenticar_api()

kmz_pathway = api.velocidade_media_cidade("BC")

# Extrai o conteúdo do arquivo KML do arquivo KMZ
kml_content = extract_kml_from_kmz(kmz_pathway)

# Lê o conteúdo do arquivo KML e extrai as descrições e coordenadas
trechos = read_kml_content(kml_content)


In [ ]:
exportar_datasets(trechos, "velocidades_médias")

In [ ]:
import pandas as pd
df_trechos = pd.DataFrame(trechos)
df_trechos